In [ ]:
# optimization for speed
# calculate first one-third, last one third to reduce workload
import pandas as pd 
import numpy as np
from itertools import combinations 
import xlrd
import pyxlsb
import datetime as dt


def right_find(policyCol, cpcAmountCol, accAmountCol):
    condition = [cpcAmountCol!=0 and accAmountCol==0 and policyCol.rfind('.')>policyCol.rfind('-'),
                cpcAmountCol!=0 and accAmountCol==0 and policyCol.rfind('.')<policyCol.rfind('-')]

    result=[policyCol[0:policyCol.rfind('.')],
            policyCol[0:policyCol.rfind('-')]]
    return np.select(condition,result,policyCol)

# Function to check if there is negative item
def check_negative(iterable):
    lst=[x for x in iterable if x<0]
    if len(lst)==1 and sum(iterable)<0:
        return False
    else:
        return True

# Function to create an iterable
def create_iterable(length):
    x=[i for i in range(length,1,-1)]
    y=[i for i in range(2,length+1)]
    z=list(zip(x,y))
    return [x for y in z for x in y][0:len(z)]

# Function to check net-off amount for Non Marine
def netOffCheckNonMarine(x,df,col,totalLimit):
    #col : columns to select
    #threshold : amount limit
    separateGroup=df.loc[x.index,col]
    separateGroup.sort_values(col)
    dict=separateGroup.to_dict()
    index_list=[]
    print('separateGroup:Checking...')
    print(separateGroup.head(1))
    print('\n')
    
    for i in create_iterable(len(dict[col[0]])):
        if len(dict[col[0]])>1 and check_negative(list(dict[col[1]].values()))==True:
            index_com=combinations(dict[col[0]].keys(),i)
            amount_com=combinations(dict[col[1]].values(),i)
            for j,k in zip(amount_com,index_com):
                # if first item of each combination > 0=> no net-off ( df is sorted ascending)
                if j[0]>0:
                    break
                #if sum of an array=0 and each array does not contain an item of another array
                if np.absolute(np.sum(np.array(j)))<=totalLimit and len([x for x in k if x in [ y for z in index_list for y in z]])==0:
                    index_list.append(k)
                    for l in k:
                        del dict[col[0]][l]
                        del dict[col[1]][l]

    index_list=[x for y in index_list for x in y]
    index_list.sort()
    index_list2=["Net-off" if i in index_list  else "" for i in x.index]
    print('Done!')
    return index_list2

def netOffCheckMarine(x,df,col,totalLimit,rowLimit):
    #col : columns to select
    #totalLimit : amount limit
    #rowLimit : total number of row to make combinations, if rowLimit> 30, if is very hard to calculate using PC
    separateGroup=df.loc[x.index,col]
    separateGroup.sort_values(col)
    dict=separateGroup.to_dict()
    index_list=[]
    print('separateGroup:Checking...')
    print(separateGroup.head(1))
    print('\n')
    for i in create_iterable(len(dict[col[0]])):
        if len(dict[col[0]])>1 and len(dict[col[0]])<rowLimit and check_negative(list(dict[col[1]].values()))==True:
            index_com=combinations(dict[col[0]].keys(),i)
            amount_com=combinations(dict[col[1]].values(),i)
            for j,k in zip(amount_com,index_com):
                  # if first item of each combination > 0=> no net-off ( df is sorted ascending)
                if j[0]>0:
                    break
                #if sum of an array=0 and each array does not contain an item of another array
                if np.absolute(np.sum(np.array(j)))<=totalLimit and len([x for x in k if x in [ y for z in index_list for y in z]])==0:
                    index_list.append(k)
                    for l in k:
                        del dict[col[0]][l]
                        del dict[col[1]][l]

    index_list=[x for y in index_list for x in y]
    index_list.sort()
    index_list2=["Net-off" if i in index_list  else "" for i in x.index]
    print('Done!')
    return index_list2

#Set threshold for Marine to filter marine policy with diff amount to 3% or bigger
marineThreshold=0.03 

# Read file ,dfMarinePolicy contains Marine Policy needs to be checked
fileLastMonth=r"D:\OneDrive\OneDrive - khoavanhoc.edu.vn\UIC\DOI CHIEU ACC-CPC\2020\01-10\Check RWP Fact 01-10 2020.xlsb"
fileThisMonth=r"D:\OneDrive\OneDrive - khoavanhoc.edu.vn\UIC\DOI CHIEU ACC-CPC\2020\01-11\Check RWP 01-11 2020.xlsx"
fileLastMonth.replace('\\','/')
fileThisMonth.replace('\\','/')

dfCheckSQL=pd.read_excel(fileThisMonth,usecols=['Nhan tai/Truc tiep','Số đơn','Số đơn đến mã phòng','Mã nghiệp vụ ','Phòng','Phí bảo hiểm NT CPC','Phí bảo hiểm NT ACC','Chênh lệch NT','Phí bảo hiểm VND CPC','Phí bảo hiểm VND ACC','Chênh lệch VND','Final Note'])
dfCheckLastMonth=pd.read_excel(fileLastMonth,sheet_name='Sheet1',usecols=['Số đơn','Final Note'],skiprows=1,engine='pyxlsb')

# Clean dfCheckSQL
#dfCheckSQL['Số đơn']=dfCheckSQL['Số đơn'].str.strip() # clean white space
#dfCheckSQL['Số đơn đến mã phòng']=dfCheckSQL['Số đơn đến mã phòng'].str.strip() # clean white space
#dfCheckSQL['Số đơn']=np.where(dfCheckSQL['Số đơn'].str.contains('UPDATE'),dfCheckSQL['Số đơn'].str.slice(start=7),dfCheckSQL['Số đơn'])
#dfCheckSQL['Số đơn đến mã phòng']=np.where(dfCheckSQL['Số đơn đến mã phòng'].str.contains('UPDATE'),dfCheckSQL['Số đơn đến mã phòng'].str.slice(start=7),dfCheckSQL['Số đơn đến mã phòng'])
#MarinePolicy=pd.read_excel('D:/Check_GWP.xlsx',sheet_name='MarinePolicy').values()

# Define col for 02 groups
colNonMarine=['Số đơn đến mã phòng','Chênh lệch VND']
colMarine=['PolicyForChecking','Chênh lệch VND']

# Create df for Non Marine
dfNonMarine=dfCheckSQL.loc[(~dfCheckSQL['Mã nghiệp vụ '].isin(['MCA','ACA','MCE','MCI','ICA','COT',' MC']))&(dfCheckSQL['Final Note']).isin(['Check','Acc chưa ghi nhận','CLTG-Cargo'])]
dfNonMarine=dfNonMarine.sort_values(colNonMarine)

dfNonMarine['Số đơn']=dfNonMarine['Số đơn'].str.strip() # clean white space
dfNonMarine['Số đơn đến mã phòng']=dfNonMarine['Số đơn đến mã phòng'].str.strip() # clean white space
dfNonMarine['Số đơn']=np.where(dfNonMarine['Số đơn'].str.contains('UPDATE'),dfNonMarine['Số đơn'].str.slice(start=7),dfNonMarine['Số đơn'])
dfNonMarine['Số đơn đến mã phòng']=np.where(dfNonMarine['Số đơn đến mã phòng'].str.contains('UPDATE'),dfNonMarine['Số đơn đến mã phòng'].str.slice(start=7),dfNonMarine['Số đơn đến mã phòng'])


# Create df for Marine
dfMarine=dfCheckSQL.loc[(dfCheckSQL['Mã nghiệp vụ '].isin(['MCA','ACA','MCE','MCI','ICA','COT',' MC']))&(dfCheckSQL['Final Note']).isin(['Check','Acc chưa ghi nhận','CLTG-Cargo'])]
#dfMarine=dfMarine.sort_values(colMarine)

# Create df group by for Marine
dfMarineGroupBy=dfMarine.groupby(['Số đơn đến mã phòng'],as_index=False)['Phí bảo hiểm VND CPC','Phí bảo hiểm VND ACC'].sum()
dfMarineGroupBy['PercentageDiff']=(2*(dfMarineGroupBy['Phí bảo hiểm VND CPC']-dfMarineGroupBy['Phí bảo hiểm VND ACC']+1)/(dfMarineGroupBy['Phí bảo hiểm VND CPC']+dfMarineGroupBy['Phí bảo hiểm VND ACC'])).abs()

marineList=dfMarineGroupBy.loc[dfMarineGroupBy['PercentageDiff']>marineThreshold,['Số đơn đến mã phòng']]
marineList=marineList['Số đơn đến mã phòng'].values

# Filter dfMarine 
dfMarineForChecking=dfMarine.loc[dfMarine['Số đơn đến mã phòng'].isin(marineList)]
dfMarineForChecking['Số đơn']=dfMarineForChecking['Số đơn'].str.strip() # clean white space
dfMarineForChecking['Số đơn đến mã phòng']=dfMarineForChecking['Số đơn đến mã phòng'].str.strip() # clean white space
dfMarineForChecking['Số đơn']=np.where(dfMarineForChecking['Số đơn'].str.contains('UPDATE'),dfMarineForChecking['Số đơn'].str.slice(start=7),dfMarineForChecking['Số đơn'])
dfMarineForChecking['Số đơn đến mã phòng']=np.where(dfMarineForChecking['Số đơn đến mã phòng'].str.contains('UPDATE'),dfMarineForChecking['Số đơn đến mã phòng'].str.slice(start=7),dfMarineForChecking['Số đơn đến mã phòng'])
dfMarineNOTForChecking=dfMarine.loc[~dfMarine['Số đơn đến mã phòng'].isin(marineList)]
dfMarineNOTForChecking['Check']=dfMarineNOTForChecking['Final Note']
dfMarineForChecking['PolicyForChecking']=dfMarineForChecking.apply(lambda x: right_find(x['Số đơn'],x['Phí bảo hiểm VND CPC'],x['Phí bảo hiểm VND ACC']),axis=1).astype(str)
dfMarineForChecking=dfMarineForChecking.sort_values(colMarine)

# Create df for the rest
dfOk=dfCheckSQL.loc[(~dfCheckSQL['Final Note'].isin(['Check','Acc chưa ghi nhận','CLTG-Cargo']))]
dfOk['Check']=dfOk['Final Note']

# Create column for Marine policy

dfNonMarine['Check']=dfNonMarine.groupby([colNonMarine[0]])[colNonMarine[1]].transform(netOffCheckNonMarine,df=dfNonMarine,col=colNonMarine,totalLimit=10000)

dfMarineForChecking['Check']=dfMarineForChecking.groupby([colMarine[0]])[colMarine[1]].transform(netOffCheckMarine,df=dfMarineForChecking,col=colMarine,totalLimit=20000,rowLimit=101)

dfCheckFinal=pd.concat([dfNonMarine,dfMarineForChecking,dfMarineNOTForChecking,dfOk],axis=0)
dfCheckFinal['Note']=np.where(dfCheckFinal['Check']=='Net-off',dfCheckFinal['Check'],dfCheckFinal['Final Note'])
dfCheckFinal=dfCheckFinal.drop(columns=['Final Note','Check'])

dfCheckLastMonth=dfCheckLastMonth.rename(columns={'Final Note':'Note thang truoc'})
dfCheckFinal=dfCheckFinal.merge(dfCheckLastMonth, on='Số đơn',how='left')
time=str(dt.datetime.now()).replace(':','.')
dfCheckFinal.to_csv(f'D:/dfCheckFinal{time}.csv',encoding='utf-8-sig')
import subprocess
subprocess.Popen([f'D:/dfCheckFinal{time}.csv'],shell=True)

